In [1]:
# Imports
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
import requests
from bs4 import BeautifulSoup
warnings.filterwarnings('ignore')

In [2]:
# Load SP500 stocks data
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = requests.get(url, verify=False)  # Set verify to False to ignore SSL certificate verification

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')
sp500 = pd.read_html(str(soup.find_all('table')[0]))[0]

#replace some symbols to use in yfinance
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

#create a list with SP500 tickers
symbols_list = sp500['Symbol'].unique().tolist()
symbols_list

#select stard and end date
end_date = '2023-09-27'
start_date = pd.to_datetime(end_date) - pd.DateOffset(365*8) # remove five years from the end_date

df = yf.download(tickers=symbols_list,
                start=start_date,
                end=end_date)
df

[*********************100%***********************]  503 of 503 completed

498 Failed downloads:
- TDG: No data found for this date range, symbol may be delisted
- NCLH: No data found for this date range, symbol may be delisted
- ALGN: No data found for this date range, symbol may be delisted
- DVA: No data found for this date range, symbol may be delisted
- EG: No data found for this date range, symbol may be delisted
- ULTA: No data found for this date range, symbol may be delisted
- PCAR: No data found for this date range, symbol may be delisted
- CEG: No data found for this date range, symbol may be delisted
- FIS: No data found for this date range, symbol may be delisted
- FDX: No data found for this date range, symbol may be delisted
- LYV: No data found for this date range, symbol may be delisted
- MKTX: No data found for this date range, symbol may be delisted
- TYL: No data found for this date range, symbol may be delisted
- PRU: No data found for this date range, symbol may be

Adj Close                                         \
                                  A AAL        AAPL ABBV ABNB ABT ACGL ACN   
Date                                                                         
2015-09-29 00:00:00-04:00       NaN NaN   25.072458  NaN  NaN NaN  NaN NaN   
2015-09-30 00:00:00-04:00       NaN NaN   25.357531  NaN  NaN NaN  NaN NaN   
2015-10-01 00:00:00-04:00       NaN NaN   25.192007  NaN  NaN NaN  NaN NaN   
2015-10-02 00:00:00-04:00       NaN NaN   25.375925  NaN  NaN NaN  NaN NaN   
2015-10-05 00:00:00-04:00       NaN NaN   25.467878  NaN  NaN NaN  NaN NaN   
...                             ...  ..         ...  ...  ...  ..  ...  ..   
2023-09-20 00:00:00-04:00       NaN NaN  175.490005  NaN  NaN NaN  NaN NaN   
2023-09-21 00:00:00-04:00       NaN NaN  173.929993  NaN  NaN NaN  NaN NaN   
2023-09-22 00:00:00-04:00       NaN NaN  174.789993  NaN  NaN NaN  NaN NaN   
2023-09-25 00:00:00-04:00       NaN NaN  176.080002  NaN  NaN NaN  NaN NaN   
2023-09-26 00:00:00-04:00       NaN NaN  171.960007  NaN  NaN NaN  NaN NaN   

                                    ... Volume                                \
                          ADBE ADI  ...   WYNN XEL XOM XRAY XYL YUM ZBH ZBRA   
Date                                ...                                        
2015-09-29 00:00:00-04:00  NaN NaN  ...    NaN NaN NaN  NaN NaN NaN NaN  NaN   
2015-09-30 00:00:00-04:00  NaN NaN  ...    NaN NaN NaN  NaN NaN NaN NaN  NaN   
2015-10-01 00:00:00-04:00  NaN NaN  ...    NaN NaN NaN  NaN NaN NaN NaN  NaN   
2015-10-02 00:00:00-04:00  NaN NaN  ...    NaN NaN NaN  NaN NaN NaN NaN  NaN   
2015-10-05 00:00:00-04:00  NaN NaN  ...    NaN NaN NaN  NaN NaN NaN NaN  NaN   
...                        ...  ..  ...    ...  ..  ..  ...  ..  ..  ..  ...   
2023-09-20 00:00:00-04:00  NaN NaN  ...    NaN NaN NaN  NaN NaN NaN NaN  NaN   
2023-09-21 00:00:00-04:00  NaN NaN  ...    NaN NaN NaN  NaN NaN NaN NaN  NaN   
2023-09-22 00:00:00-04:00  NaN NaN  ...    NaN NaN NaN  NaN NaN NaN NaN  NaN   
2023-09-25 00:00:00-04:00  NaN NaN  ...    NaN NaN NaN  NaN NaN NaN NaN  NaN   
2023-09-26 00:00:00-04:00  NaN NaN  ...    NaN NaN NaN  NaN NaN NaN NaN  NaN   

                                    
                          ZION ZTS  
Date                                
2015-09-29 00:00:00-04:00  NaN NaN  
2015-09-30 00:00:00-04:00  NaN NaN  
2015-10-01 00:00:00-04:00  NaN NaN  
2015-10-02 00:00:00-04:00  NaN NaN  
2015-10-05 00:00:00-04:00  NaN NaN  
...                        ...  ..  
2023-09-20 00:00:00-04:00  NaN NaN  
2023-09-21 00:00:00-04:00  NaN NaN  
2023-09-22 00:00:00-04:00  NaN NaN  
2023-09-25 00:00:00-04:00  NaN NaN  
2023-09-26 00:00:00-04:00  NaN NaN  

[2012 rows x 3018 columns]

In [3]:
#Organize our indexexaset to be more clear
df = df.stack()
df.index.names = ['date', 'ticker']
df.columns = df.columns.str.lower()
df

adj close       close        high  \
date                      ticker                                       
2015-09-29 00:00:00-04:00 AAPL     25.072458   27.264999   28.377501   
                          GOOGL    31.130501   31.130501   31.733999   
                          META     86.669998   86.669998   90.059998   
                          MSFT     39.194000   43.439999   43.570000   
                          TSLA     16.443333   16.443333   16.982000   
...                                      ...         ...         ...   
2023-09-26 00:00:00-04:00 AAPL    171.960007  171.960007  175.199997   
                          GOOGL   128.570007  128.570007  130.360001   
                          META    298.959991  298.959991  300.299988   
                          MSFT    312.140015  312.140015  315.880005   
                          TSLA    244.119995  244.119995  249.550003   

                                         low        open       volume  
date                      ticker                                       
2015-09-29 00:00:00-04:00 AAPL     26.965000   28.207500  293461600.0  
                          GOOGL    30.901501   31.275000   42130000.0  
                          META     85.720001   89.099998   42281800.0  
                          MSFT     43.049999   43.369999   32763600.0  
                          TSLA     16.364000   16.697332   55548000.0  
...                                      ...         ...          ...  
2023-09-26 00:00:00-04:00 AAPL    171.660004  174.820007   64588900.0  
                          GOOGL   127.220001  129.770004   25718700.0  
                          META    296.010010  297.660004   19417200.0  
                          MSFT    310.019989  315.130005   26297600.0  
                          TSLA    241.660004  242.979996  101993600.0  

[10060 rows x 6 columns]

### Calculate technical indicators

In [19]:
#Garman_klass Volatility
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

#RSI
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))
#df.xs('AAPL', level=1)['rsi'].plot() # plot rsi of apple

# BollingerBands
df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

#ATR
def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                 low=stock_data['low'],
                 close=stock_data['close'],
                 lenght=14)
    return atr.sub(atr.mean()).div(atr.std())


df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

#MACD
def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std()) #calutation to normalize


df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

#Calculate dollar volume
df['dollar_volume'] = (df['adj close'] * df['volume']) / 1e6 #divide by 1 milion because milions of shares are trated daily

df

adj close       close        high  \
date                      ticker                                       
2015-09-29 00:00:00-04:00 AAPL     25.072458   27.264999   28.377501   
                          GOOGL    31.130501   31.130501   31.733999   
                          META     86.669998   86.669998   90.059998   
                          MSFT     39.194000   43.439999   43.570000   
                          TSLA     16.443333   16.443333   16.982000   
...                                      ...         ...         ...   
2023-09-26 00:00:00-04:00 AAPL    171.960007  171.960007  175.199997   
                          GOOGL   128.570007  128.570007  130.360001   
                          META    298.959991  298.959991  300.299988   
                          MSFT    312.140015  312.140015  315.880005   
                          TSLA    244.119995  244.119995  249.550003   

                                         low        open       volume  \
date                      ticker                                        
2015-09-29 00:00:00-04:00 AAPL     26.965000   28.207500  293461600.0   
                          GOOGL    30.901501   31.275000   42130000.0   
                          META     85.720001   89.099998   42281800.0   
                          MSFT     43.049999   43.369999   32763600.0   
                          TSLA     16.364000   16.697332   55548000.0   
...                                      ...         ...          ...   
2023-09-26 00:00:00-04:00 AAPL    171.660004  174.820007   64588900.0   
                          GOOGL   127.220001  129.770004   25718700.0   
                          META    296.010010  297.660004   19417200.0   
                          MSFT    310.019989  315.130005   26297600.0   
                          TSLA    241.660004  242.979996  101993600.0   

                                  garman_klass_vol        rsi    bb_low  \
date                      ticker                                          
2015-09-29 00:00:00-04:00 AAPL           -0.004059        NaN       NaN   
                          GOOGL           0.000345        NaN       NaN   
                          META            0.000924        NaN       NaN   
                          MSFT           -0.003888        NaN       NaN   
                          TSLA            0.000596        NaN       NaN   
...                                            ...        ...       ...   
2023-09-26 00:00:00-04:00 AAPL            0.000103  40.386252  5.134923   
                          GOOGL           0.000264  43.997716  4.871573   
                          META            0.000096  50.262069  5.681562   
                          MSFT            0.000140  38.358054  5.753026   
                          TSLA            0.000508  45.955272  5.480648   

                                    bb_mid   bb_high       atr      macd  \
date                      ticker                                           
2015-09-29 00:00:00-04:00 AAPL         NaN       NaN       NaN       NaN   
                          GOOGL        NaN       NaN       NaN       NaN   
                          META         NaN       NaN       NaN       NaN   
                          MSFT         NaN       NaN       NaN       NaN   
                          TSLA         NaN       NaN       NaN       NaN   
...                                    ...       ...       ...       ...   
2023-09-26 00:00:00-04:00 AAPL    5.194552  5.254180  0.967273 -1.414184   
                          GOOGL   4.912076  4.952579  0.930630 -0.192760   
                          META    5.708585  5.735609  0.961079 -0.040023   
                          MSFT    5.796374  5.839722  0.894571 -1.317375   
                          TSLA    5.558842  5.637036  0.824771 -0.027720   

                                  dollar_volume  
date                      ticker                 
2015-09-29 00:00:00-04:00 AAPL      7357.803719  
                          GOOGL  

### Aggregate to monthly level and filter mos liquid stock

In [22]:
df.unstack('ticker')['dollar_volume'].resample('M').mean() #agregate by month

ticker,AAPL,GOOGL,META,MSFT,TSLA
date,,,,,
2015-09-30 00:00:00-04:00,7050.084122,1500.870148,3458.082837,1340.090063,1068.953422
2015-10-31 00:00:00-04:00,5285.594113,1584.213103,2526.490236,1748.174716,1017.910952
2015-11-30 00:00:00-05:00,4092.760227,1426.139264,2901.607961,1615.109580,868.514543
2015-12-31 00:00:00-05:00,4319.932957,1527.790769,2106.832258,1808.617125,627.100746
2016-01-31 00:00:00-05:00,6067.452374,2015.825533,4172.506229,2326.067193,855.033458
...,...,...,...,...,...
2023-05-31 00:00:00-04:00,10010.490581,4357.569070,5400.470759,8631.874445,21573.416958
2023-06-30 00:00:00-04:00,11372.140753,3830.668431,6367.596815,8737.577636,40338.857362
2023-07-31 00:00:00-04:00,9582.336256,4371.732062,9483.127463,11434.356688,32726.593194


In [26]:
#Create dict with indicators columns
indicator_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume',
                                                              'open', 'close', 'high',
                                                              'low']]
indicator_cols

['adj close',
 'garman_klass_vol',
 'rsi',
 'bb_low',
 'bb_mid',
 'bb_high',
 'atr',
 'macd']

In [23]:
df.unstack('ticker')['dollar_volume'].resample('M').mean().stack().to_frame('dollar_volume')

dollar_volume
date                      ticker               
2015-09-30 00:00:00-04:00 AAPL      7050.084122
                          GOOGL     1500.870148
                          META      3458.082837
                          MSFT      1340.090063
                          TSLA      1068.953422
...                                         ...
2023-09-30 00:00:00-04:00 AAPL     12146.172063
                          GOOGL     3181.391274
                          META      5714.408578
                          MSFT      6804.980194
                          TSLA     31386.906936

[485 rows x 1 columns]

In [28]:
df.unstack()[indicator_cols].resample('M').last().stack('ticker')

adj close  garman_klass_vol        rsi  \
date                      ticker                                            
2015-09-30 00:00:00-04:00 AAPL     25.357531         -0.002314        NaN   
                          GOOGL    31.918501          0.000087        NaN   
                          META     89.900002          0.000151        NaN   
                          MSFT     39.933853         -0.003324        NaN   
                          TSLA     16.559999          0.000747        NaN   
...                                      ...               ...        ...   
2023-09-30 00:00:00-04:00 AAPL    171.960007          0.000103  40.386252   
                          GOOGL   128.570007          0.000264  43.997716   
                          META    298.959991          0.000096  50.262069   
                          MSFT    312.140015          0.000140  38.358054   
                          TSLA    244.119995          0.000508  45.955272   

                                    bb_low    bb_mid   bb_high       atr  \
date                      ticker                                           
2015-09-30 00:00:00-04:00 AAPL         NaN       NaN       NaN       NaN   
                          GOOGL        NaN       NaN       NaN       NaN   
                          META         NaN       NaN       NaN       NaN   
                          MSFT         NaN       NaN       NaN       NaN   
                          TSLA         NaN       NaN       NaN       NaN   
...                                    ...       ...       ...       ...   
2023-09-30 00:00:00-04:00 AAPL    5.134923  5.194552  5.254180  0.967273   
                          GOOGL   4.871573  4.912076  4.952579  0.930630   
                          META    5.681562  5.708585  5.735609  0.961079   
                          MSFT    5.753026  5.796374  5.839722  0.894571   
                          TSLA    5.480648  5.558842  5.637036  0.824771   

                                      macd  
date                      ticker            
2015-09-30 00:00:00-04:00 AAPL         NaN  
                          GOOGL        NaN  
                          META         NaN  
                          MSFT         NaN  
                          TSLA         NaN  
...                                    ...  
2023-09-30 00:00:00-04:00 AAPL   -1.414184  
                          GOOGL  -0.192760  
                          META   -0.040023  
                          MSFT   -1.317375  
                          TSLA   -0.027720  

[485 rows x 8 columns]

In [34]:
#Combine indicators with dollar_volume mean by month
data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack().to_frame('dollar_volume'),
          df.unstack()[indicator_cols].resample('M').last().stack('ticker')],
          axis=1)).dropna()

data

dollar_volume   adj close  garman_klass_vol  \
date                      ticker                                                
2015-11-30 00:00:00-05:00 AAPL      4092.760227   27.313120         -0.002189   
                          GOOGL     1426.139264   38.142502          0.000111   
                          META      2901.607961  104.239998          0.000161   
                          MSFT      1615.109580   49.368141         -0.003679   
                          TSLA       868.514543   15.350667          0.000235   
...                                         ...         ...               ...   
2023-09-30 00:00:00-04:00 AAPL     12146.172063  171.960007          0.000103   
                          GOOGL     3181.391274  128.570007          0.000264   
                          META      5714.408578  298.959991          0.000096   
                          MSFT      6804.980194  312.140015          0.000140   
                          TSLA     31386.906936  244.119995          0.000508   

                                        rsi    bb_low    bb_mid   bb_high  \
date                      ticker                                            
2015-11-30 00:00:00-05:00 AAPL    55.537386  3.297995  3.341333  3.384671   
                          GOOGL   63.168941  3.637679  3.663018  3.688358   
                          META    55.404379  4.638016  4.672391  4.706767   
                          MSFT    67.377488  3.887128  3.907274  3.927421   
                          TSLA    54.242021  2.687592  2.754718  2.821844   
...                                     ...       ...       ...       ...   
2023-09-30 00:00:00-04:00 AAPL    40.386252  5.134923  5.194552  5.254180   
                          GOOGL   43.997716  4.871573  4.912076  4.952579   
                          META    50.262069  5.681562  5.708585  5.735609   
                          MSFT    38.358054  5.753026  5.796374  5.839722   
                          TSLA    45.955272  5.480648  5.558842  5.637036   

                                       atr      macd  
date                      ticker                      
2015-11-30 00:00:00-05:00 AAPL   -0.967900 -0.139221  
                          GOOGL  -1.033524  0.375191  
                          META   -1.165352  0.202517  
                          MSFT   -1.056035  0.064642  
                          TSLA   -0.809146 -0.114649  
...                                    ...       ...  
2023-09-30 00:00:00-04:00 AAPL    0.967273 -1.414184  
                          GOOGL   0.930630 -0.192760  
                          META    0.961079 -0.040023  
                          MSFT    0.894571 -1.317375  
                          TSLA    0.824771 -0.027720  

[475 rows x 9 columns]